In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.preprocessing import scale
from statsmodels.base.model import GenericLikelihoodModel
%matplotlib inline

In [37]:
def splitXY(dfXY):
    lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
    dfX = dfXY.drop(lbls, axis=1)
    if 'total' in dfX.columns:
        dfX.drop('total', axis=1, inplace=True)
    r_dfY = dfXY.loc[:, lbls[0]]
    c_dfY = dfXY.loc[:, lbls[1]]
    e_dfY = dfXY.loc[:, lbls[2]]
    b_dfY = dfXY.loc[:, lbls[3]]
    o_dfY = dfXY.loc[:, lbls[4]]
    return dfX, r_dfY, c_dfY, e_dfY, b_dfY, o_dfY

In [38]:
pklfile = '../learn/pkl_trainsets/28jun2019/_nucs_fissact_not-scaled.pkl'
trainXY = pd.read_pickle(pklfile)
trainXY.reset_index(inplace=True, drop=True)
trainXY = trainXY.sample(frac=0.5)
X, rY, cY, eY, bY, oY = splitXY(trainXY)
#X = scale(X)

# ML Calculation
## Likelihoods of each observation calculated for a given input

# ML Estimation
## Coefficients of linear models estimated
This example is done with dependent variable = burnup

In [36]:
class NormMod(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(NormMod, self).__init__(endog, exog, **kwds)
    
    def nloglikeobs(self, params):
        beta = params
        yhat = np.dot(self.exog, beta)
        LL = stats.norm.logpdf(self.endog, loc=yhat)
        return -LL
    
    def fit(self, start_params=None, maxiter=100000, maxfun=100000, **kwds):
        if start_params == None:
            start_params = np.zeros(self.exog.shape[1])
            start_params[0] = np.log(self.endog.mean())
        return super(NormMod, self).fit(start_params=start_params,
                                        maxiter=maxiter, maxfun=maxfun,
                                        **kwds)

In [39]:
y = bY
X = sm.add_constant(X)

/home/opotowsky/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [40]:
X.shape

(12398, 48)

In [41]:
mod = NormMod(y, X)

In [42]:
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 267306.240103
         Iterations: 36198
         Function evaluations: 41431
                               NormMod Results                                
Dep. Variable:                 Burnup   Log-Likelihood:            -3.3141e+09
Model:                        NormMod   AIC:                         6.628e+09
Method:            Maximum Likelihood   BIC:                         6.628e+09
Date:                Wed, 14 Aug 2019                                         
Time:                        14:43:16                                         
No. Observations:               12398                                         
Df Residuals:                   12350                                         
Df Model:                          47                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

/home/opotowsky/.local/lib/python3.6/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/opotowsky/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/opotowsky/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/opotowsky/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
